<a href="https://colab.research.google.com/github/ajayrfhp/LearningDeepLearning/blob/main/imagenette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l
!pip install --upgrade torchvision

- Measure performance of LeNet and alexNet on the imagenette dataset

In [ ]:
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

import torchvision
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import os
from matplotlib import pyplot as plt


device_type = 'cpu'
if torch.cuda.is_available():
  device_type = 'cuda'

device = torch.device(device_type)
device

device(type='cuda')

## Convert pytorch data loader into d2l data module

In [ ]:
class ImageNette(d2l.DataModule):
    """ImageNette d2l module.

    Defined in :numref:`sec_fashion_mnist`"""
    def __init__(self, batch_size=128, resize=(224, 224), download=True):
        super().__init__()
        self.train_root = "../train_data/"
        self.val_root = "../val_data/"
        self.save_hyperparameters()
        transform= transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(resize),
        transforms.ToTensor()])
        self.train = datasets.Imagenette(root=self.train_root, split='train', download=download, transform=transform)
        self.val = datasets.Imagenette(root=self.val_root, split='val', download=download, transform=transform)
        self.labels = self.train.classes

    def text_labels(self, indices):
        """Return text labels.

        Defined in :numref:`sec_fashion_mnist`"""
        return [self.labels[int(i)] for i in indices]

    def get_dataloader(self, train):
        """Defined in :numref:`sec_fashion_mnist`"""
        data = self.train if train else self.val
        return torch.utils.data.DataLoader(data, self.batch_size, shuffle=train,
                                           num_workers=self.num_workers)

    def visualize(self, batch, nrows=1, ncols=8, labels=[]):
        """Defined in :numref:`sec_fashion_mnist`"""
        X, y = batch
        if not labels:
            labels = self.text_labels(y)
        d2l.show_images(X.squeeze(1), nrows, ncols, titles=labels)

In [ ]:
!rm -rf ../train_data/imagenette2/
!rm -rf ../val_data/imagenette2/

In [ ]:
data = ImageNette(batch_size=1028, download=True)
data

100%|██████████| 1557161267/1557161267 [00:45<00:00, 34066728.78it/s]


Extracting ../train_data/imagenette2.tgz to ../train_data/


100%|██████████| 1557161267/1557161267 [00:29<00:00, 53421310.42it/s]


Extracting ../val_data/imagenette2.tgz to ../val_data/


In [ ]:
def init_cnn(module):
    """Initialize weights for CNNs."""
    if type(module) == nn.LazyLinear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)

class LeNetModern(d2l.Classifier):
  def __init__(self, num_classes=10, lr=0.1):
    super().__init__()
    self.lr = lr
    self.net = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=6, out_channels=6, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Flatten(),
        nn.LazyLinear(120),
        nn.ReLU(),
        nn.LazyLinear(84),
        nn.ReLU(),
        nn.LazyLinear(num_classes)
    )

  def forward(self, x):
    return self.net(x)

  def layer_summary(self, X_shape):
    X = torch.randn(*X_shape)
    total_params = 0
    for layer in self.net:
      X = layer(X)
      params = sum([p.numel() for p in layer.parameters()])
      print(layer.__class__.__name__, "output shape = ", X.shape, "params = ", params)
      total_params += params
    print("Total params In M = ", total_params / (1e6))



modern_model = LeNetModern()
modern_model.layer_summary((1, 3, 224, 224))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Conv2d output shape =  torch.Size([1, 6, 224, 224]) params =  456
ReLU output shape =  torch.Size([1, 6, 224, 224]) params =  0
MaxPool2d output shape =  torch.Size([1, 6, 112, 112]) params =  0
Conv2d output shape =  torch.Size([1, 6, 112, 112]) params =  906
ReLU output shape =  torch.Size([1, 6, 112, 112]) params =  0
MaxPool2d output shape =  torch.Size([1, 6, 56, 56]) params =  0
Conv2d output shape =  torch.Size([1, 16, 52, 52]) params =  2416
ReLU output shape =  torch.Size([1, 16, 52, 52]) params =  0
MaxPool2d output shape =  torch.Size([1, 16, 26, 26]) params =  0
Flatten output shape =  torch.Size([1, 10816]) params =  0
Linear output shape =  torch.Size([1, 120]) params =  1298040
ReLU output shape =  torch.Size([1, 120]) params =  0
Linear output shape =  torch.Size([1, 84]) params =  10164
ReLU output shape =  torch.Size([1, 84]) params =  0
Linear output shape =  torch.Size([1, 10]) params =  850
Total params In M =  1.312832


In [ ]:

def get_loss(model, data):
  model.train = False

  losses, access = [], []
  for (inputs, outputs) in data.val_dataloader():
    preds = model.forward(inputs.to(device))
    loss = model.loss(preds, outputs.to(device)).item()
    acc = model.accuracy(preds, outputs.to(device)).item()
    losses.append(loss)
    access.append(acc)

  return np.mean(np.array(losses)), np.mean(np.array(access))

In [ ]:
trainer = d2l.Trainer(max_epochs=200, num_gpus=1)
le_net_modern = LeNetModern(lr=0.1)
le_net_modern.apply_init([next(iter(data.get_dataloader(True)))[0]], init_cnn)
trainer.fit(le_net_modern, data)
get_loss(le_net_modern, data)

In [ ]:
from google.colab import files
import helper

torch.save(le_net_modern.state_dict(), 'le_net_modern.pth')
files.download('le_net_modern.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!nvidia-smi

Wed Mar 13 22:10:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              39W / 300W |  13952MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--